In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
import requests
import os
from splinter import Browser


In [2]:
executable_path = {'executable_path' : ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\katie\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [3]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# NASA Mars News

mars = {}

results = soup.find_all('div', class_='content_title')
title = results[1].a.text

results = soup.find_all('div', class_= 'article_teaser_body')
para = results[0].text

mars['news_title'] = title
mars['news_paragraph'] = para


In [5]:
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'



In [6]:
# JPL Mars Space Image

browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

results = browser.find_by_tag('button')[1]
results.click()
img_results = soup.find_all('img')[1]['src']
mars['featured_image_url'] = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/' + img_results


In [7]:
url = 'https://space-facts.com/mars/'

In [8]:
# Mars Facts

table = pd.read_html(url)[0]
table.columns=['Description', 'Mars']
table.set_index('Description', inplace = True)
mars_facts = table.to_html()
mars['mars_facts'] = mars_facts

In [9]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [11]:
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
results = soup.find_all('div', class_ = 'description')

img_urls = []

for name in results:
    dict_for_mars = {}
    temp = name.find('h3').text
    two_words = temp[:-9]
    dict_for_mars['title'] = two_words

    browser.click_link_by_partial_text(temp)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    img = soup.find_all('img')[5]['src']
    base_url = 'astrogeology.usgs.gov'
    img_url = base_url + img
    dict_for_mars['img_url'] = img
    
    img_urls.append(dict_for_mars)
    
    browser.back()
    
img_urls   

    

[{'title': 'Cerberus Hemisphere',
  'img_url': '/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': '/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': '/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': '/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [12]:
# move to the end!!

browser.quit()